# READ ME

This file process each city per run. You need first to find the URL of the webpage that generates the list of locais by navigating into the TRE-RS website and selecting the city. Then copy the URL and paste it in the "Download file from TRE-RS. This repository contains the HTML file for Porto Alegre.

As of today October 29th, one day before the second run, the website is returning an error code...

## Import libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import re

### Download file from tre-rs

Retrieve file from TRE-RS server.  When the website is functional, you should navigate there and selec the city you want to decode, and then copy the URL here.

In [2]:
url_to_decode = "https://eleitor.tre-rs.jus.br/locais-votacao/locais?q%5Bzona_numero_eq%5D=&q%5Bmunicipio_id_eq%5D=7994&button="
city_name = 'Porto Alegre'
# urllib.request.urlretrieve(url_to_decode, f"HTML files/Locais{city_name}2022.html")


### Import HTML into BeautifulSoup

In [3]:
html_soup = BeautifulSoup(open('HTML files/LocaisPortoAlegre2022.html'), "html.parser")

#### Create a dictionary with the Zonas Eleitorais and their position in the HTML code so the Secoes can be tied to the Zonas

In [4]:
dict_zonas = {}
div_content = html_soup.findAll(id= 'content')    
for div in div_content:
    for zona in div.find_all("h4"):
        zona_name = zona.text
        zona_line = zona.sourceline
        dict_zonas[zona_name] = zona_line

#### Create a DataFrame to be populated with the Zonas and Secoes information

In [5]:
secao_nome,lst_secoes,df_zonas,secao_endereco = "","","",""
columns = ['Zona', 'Local', 'Endereco', 'Secoes','Eleitores','Cidade']
df_zonas = pd.DataFrame(columns=columns)
div_content = html_soup.findAll(id= 'content')    
for div in div_content:
    for table in div.find_all("table"):
        for tr in table.find_all("tr"):
            td_c = 0
            for td in tr.findAll("td"):
                td_c+=1
                if td_c ==1:
                    # This is the first cell of the Secoes table, it contains the name and address of the secao place. 
                    # The information is split by a new line ('\n'), so I split by new line
                    secao_nome_endereco_lst = td.text.split("\n")
                    # Both name and address (positions 1 and 2 in the list) has extra blank characters, 
                    # used strip to clean the variable
                    secao_nome = secao_nome_endereco_lst[1].strip()
                    secao_endereco = secao_nome_endereco_lst[2].strip()
                    html_pos = td.sourceline
                    for i in dict_zonas:
                        if html_pos >= dict_zonas[i]:
                            numero_zona = i
                elif td_c == 2:
                    lst_secoes = []
                    #The second cell contains the list of secoes
                    secoes = td.find_all('span')
                    for s in secoes:
                        #print(s.text)
                        lst_secoes.append(s.text)
                elif td_c == 3:
                    secao_eleitores = td.text
                elif td_c == 4:
                    secao_cidade = td.text
            # print(secao_nome,lst_secoes)
            if secao_nome:
                df_zonas.loc[len(df_zonas.index)] = [numero_zona, secao_nome, secao_endereco, lst_secoes,secao_eleitores,secao_cidade]



In [6]:
df_zonas

,Zona,Local,Endereco,Secoes,Eleitores,Cidade
0,1º Zona Eleitoral,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO","[16, 60, 89, 121, 148, 179, 208, 221, 244, 320]",3969,Porto Alegre
1,1º Zona Eleitoral,COLEGIO LA SALLE DORES,"RUA RIACHUELO, 800, CENTRO HISTÓRICO","[4, 23, 50, 53, 63, 92, 93, 123, 136, 150, 155...",5897,Porto Alegre
2,1º Zona Eleitoral,COLEGIO ROMANO SANTA MARTA,"RUA MARCILIO DIAS, 1201, MENINO DEUS","[7, 68, 240, 318]",1529,Porto Alegre
3,1º Zona Eleitoral,ESCOLA DE ADMINISTRACAO DA UFRGS,"RUA WASHINGTON LUIZ, 855, CENTRO HISTÓRICO","[12, 71, 226, 247, 264]",1910,Porto Alegre
4,1º Zona Eleitoral,ESCOLA ESTADUAL AFONSO EMILIO MASSOT,"RUA JOSE HONORATO DOS SANTOS, 101, CIDADE BAIXA","[6, 9, 79, 112, 134, 200, 207, 254]",2092,Porto Alegre
...,...,...,...,...,...,...
345,161º Zona Eleitoral,ESCOLA DE EDUCAÇÃO INFANTIL ARCO IRIS II,"RUA TOBAGO, 1258 (RESTINGA VELHA), RESTINGA",[51],238,Porto Alegre
346,161º Zona Eleitoral,ESPORTE CLUBE LAJEADO,"AV. EDGAR PIRES DE CASTRO, 9316, LAGEADO","[74, 95, 425]",1191,Porto Alegre
347,161º Zona Eleitoral,PARÓQUIA N. S. APARECIDA - C. SOCIAL PE. PEDRO...,"ESTR. CHÁCARA DO BANCO, 71 (RESTINGA VELHA), R...","[63, 156]",648,Porto Alegre
348,161º Zona Eleitoral,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA","[28, 38, 123, 137, 148, 159, 181]",2297,Porto Alegre


In [7]:
def split_zonas(txt):
    return re.findall(r'\d+', txt)[0]

In [8]:
df_zonas['Zona_int'] = df_zonas['Zona'].apply(split_zonas)
df_zonas

,Zona,Local,Endereco,Secoes,Eleitores,Cidade,Zona_int
0,1º Zona Eleitoral,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO","[16, 60, 89, 121, 148, 179, 208, 221, 244, 320]",3969,Porto Alegre,1
1,1º Zona Eleitoral,COLEGIO LA SALLE DORES,"RUA RIACHUELO, 800, CENTRO HISTÓRICO","[4, 23, 50, 53, 63, 92, 93, 123, 136, 150, 155...",5897,Porto Alegre,1
2,1º Zona Eleitoral,COLEGIO ROMANO SANTA MARTA,"RUA MARCILIO DIAS, 1201, MENINO DEUS","[7, 68, 240, 318]",1529,Porto Alegre,1
3,1º Zona Eleitoral,ESCOLA DE ADMINISTRACAO DA UFRGS,"RUA WASHINGTON LUIZ, 855, CENTRO HISTÓRICO","[12, 71, 226, 247, 264]",1910,Porto Alegre,1
4,1º Zona Eleitoral,ESCOLA ESTADUAL AFONSO EMILIO MASSOT,"RUA JOSE HONORATO DOS SANTOS, 101, CIDADE BAIXA","[6, 9, 79, 112, 134, 200, 207, 254]",2092,Porto Alegre,1
...,...,...,...,...,...,...,...
345,161º Zona Eleitoral,ESCOLA DE EDUCAÇÃO INFANTIL ARCO IRIS II,"RUA TOBAGO, 1258 (RESTINGA VELHA), RESTINGA",[51],238,Porto Alegre,161
346,161º Zona Eleitoral,ESPORTE CLUBE LAJEADO,"AV. EDGAR PIRES DE CASTRO, 9316, LAGEADO","[74, 95, 425]",1191,Porto Alegre,161
347,161º Zona Eleitoral,PARÓQUIA N. S. APARECIDA - C. SOCIAL PE. PEDRO...,"ESTR. CHÁCARA DO BANCO, 71 (RESTINGA VELHA), R...","[63, 156]",648,Porto Alegre,161
348,161º Zona Eleitoral,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA","[28, 38, 123, 137, 148, 159, 181]",2297,Porto Alegre,161


In [9]:
df_zonas.to_csv("Output/Zonas and Secoes Porto Alegre.csv")

In [10]:
df_zonas_norm = pd.DataFrame()
columns = ['Zona', 'Local', 'Endereco', 'Secoes','Eleitores','Cidade']
df_zonas_norm = pd.DataFrame(columns=columns)
for index, row in df_zonas.iterrows():
    for i in row['Secoes']:
        df_zonas_norm.loc[len(df_zonas_norm.index)] = [row['Zona_int'],row['Local'],row['Endereco'],i,row['Eleitores'],row['Cidade']]
df_zonas_norm

,Zona,Local,Endereco,Secoes,Eleitores,Cidade
0,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",16,3969,Porto Alegre
1,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",60,3969,Porto Alegre
2,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",89,3969,Porto Alegre
3,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",121,3969,Porto Alegre
4,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",148,3969,Porto Alegre
...,...,...,...,...,...,...
3142,161,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA",148,2297,Porto Alegre
3143,161,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA",159,2297,Porto Alegre
3144,161,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA",181,2297,Porto Alegre
3145,161,SÃO FRANCISCO PROJETO SOCIAL,"AV. PRINCIPAL DA PONTA GROSSA, 1701, PONTA GROSSA",72,531,Porto Alegre


In [11]:
df_zonas_norm

,Zona,Local,Endereco,Secoes,Eleitores,Cidade
0,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",16,3969,Porto Alegre
1,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",60,3969,Porto Alegre
2,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",89,3969,Porto Alegre
3,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",121,3969,Porto Alegre
4,1,COLEGIO BOM JESUS - SEVIGNE,"RUA DUQUE DE CAXIAS,1443, CENTRO HISTÓRICO",148,3969,Porto Alegre
...,...,...,...,...,...,...
3142,161,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA",148,2297,Porto Alegre
3143,161,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA",159,2297,Porto Alegre
3144,161,SOCIEDADE HÍPICA PORTO ALEGRENSE,"AV. NOSSA SENHORA DE GUADALUPE, 249-339, HÍPICA",181,2297,Porto Alegre
3145,161,SÃO FRANCISCO PROJETO SOCIAL,"AV. PRINCIPAL DA PONTA GROSSA, 1701, PONTA GROSSA",72,531,Porto Alegre


In [12]:
df_zonas_norm.to_csv("Output/Zonas and Secoes Porto Alegre NORMALIZED.csv")

### Backup - Old deprecated code that served as inspiration for the final version

It is impossible to reach good results without failing a lot. Here are my failed attempts:

In [9]:
# columns = ['Zona', 'Local', 'Endereco', 'Secoes','Eleitores','Cidade']
# df_zonas = ""
# df_zonas = pd.DataFrame(columns=columns)
# i=0
# lst_secoes = []
# # for tr in html_soup.select('table'):
# #     i+=1
# #     j=0

# for str in tr.select('tr'):
#     k=0
#     for td in str.select('td'):
#         k+=1
#         #print(td)
#         # print(k)
#         if k ==1:
#             local_secao, endereco_secao = "",""
#             # This is the first cell of the Secoes table, it contains the name and address of the secao place. 
#             # The information is split by a new line ('\n'), so I split by new line
#             secao_nome_endereco_lst = td.text.split("\n")
#             # Both name and address (positions 1 and 2 in the list) has extra blank characters, 
#             # used strip to clean the variable
#             secao_nome = secao_nome_endereco_lst[1].strip()
#             secao_endereco = secao_nome_endereco_lst[2].strip()
#             print(secao_nome)
#         elif k == 2:
#             lst_secoes = []
#             #The second cell contains the list of secoes
#             secoes = td.find_all('span')
#             for s in secoes:
#                 #print(s.text)
#                 lst_secoes.append(s.text)
#     df_zonas.loc[len(df_zonas.index)] = ['Zona', secao_nome, secao_endereco, lst_secoes,'Eleitores','Cidade']

In [10]:
# def create_df_1():
#     columns = ['Zona', 'Local', 'Endereco', 'Secoes','Eleitores','Cidade']
#     df_zonas = pd.DataFrame(columns=columns)
#     for tr in html_soup.select('table'):
#         i=0
#         for str in tr.select('tr'):
#             local_secao, endereco_secao, lst_secoes = "","",""
#             for line in str:
#                 #print(len(line),line)
#                     # print(line.select_one('span'))
#                 this_line = line.text.replace('\n', '|').split('|')
#                 int_this_l = int(len(this_line))
#                 #print(int_this_l,line.text)
#                 if int_this_l == 4 and len(this_line[1]) > 3:
#                     local_secao = this_line[1][12:]
#                     # print(local_secao)
#                     endereco_secao = this_line[2][14:]
#                     # print(endereco_secao)
#                 elif (int_this_l == 3 and len(this_line[1]) < 4):
#                     secoes = line.find('span')
#                     if secoes != -1 and secoes is not None:
#                         lst_secoes = []
#                         # print(secoes.text)
#                         lst_secoes.append(secoes.text)
#                 elif (int_this_l >= 4):
#                     secoes = line.find_all('span')
#                     lst_secoes = []
#                     for s in secoes:
#                         # print(s.text)
#                         lst_secoes.append(s.text)
#                 elif (int_this_l == 1):
#                     pass
#                     #print(line.text)
#             if len(local_secao) > 0:
#                 df_zonas.loc[len(df_zonas.index)] = ['Zona', local_secao, endereco_secao, lst_secoes,'Eleitores','Cidade']
#     return(df_zonas)